### Seleccion de datos (filtrando por comunidad) para posterior visualizacion en neo4j y bloom 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from pyspark_dist_explore import hist
import matplotlib.pyplot as plt
from pyspark.sql.types import StringType,TimestampType


In [2]:
spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

warnings.filterwarnings('ignore', category=DeprecationWarning)
sc=spark.sparkContext
sc.setLogLevel ('ERROR')
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")


24/03/11 17:31:01 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/03/11 17:31:01 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 17:31:01 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/03/1

In [3]:
contaminados = spark.read.options(header=True,inferSchema=True,delimiter=",").csv("/home/cdsw/data/processed/contaminados_processed_iva_representante_total_with_labels.csv")
contaminados.createOrReplaceTempView("contaminados")

In [4]:
## Se selecciona en esta caso de la data que contiene 3 comunidades (escogidas como ejemplo)

In [5]:
seleccion=spark.sql("select * from contaminados where comunidad in (18856,21552,24949) ")
seleccion.show()
seleccion.createOrReplaceTempView("seleccion")

+--------------------+------------------+--------------+-----------+--------------------+------------+--------+--------------+---------+-----------------+
|            cont_rut|             score|total_pago_f29|   IVA_neto|     unidad_regional|n_documentos|lifetime|alerta_inicial|comunidad|      modularidad|
+--------------------+------------------+--------------+-----------+--------------------+------------+--------+--------------+---------+-----------------+
|AaF9igyV0h/q8dpZo...| 0.020976488828855|          null|   32894369|    SANTIAGO ORIENTE|        null|    null|             0|    18856|0.789362584526045|
|jQZMX1akk5R+/q6gn...|0.0413690502565515|          null|     175630|                null|        null|    null|             0|    24949|0.789362584526045|
|NE3RTDcWkhBInCQkt...|0.0192375917968015|           0.0|   55890027|   SANTIAGO PONIENTE|        null|   842.0|             0|    18856|0.789362584526045|
|Jn46LZRg9mBct7sSJ...|0.0413690502565515|          null|      79832|  

In [6]:
# Guardamos los constribuyentes involucrados en esas comunidades

In [7]:
seleccion=seleccion.toPandas()
#guardamos los valores de contaminacion con las columnas adicionales
seleccion.to_csv('/home/cdsw/data/processed/muestra_visualizacion/contaminados_processed_iva_representante_total_with_labels_FILTERED.csv', index=False)


In [8]:
spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/propuesta_f29_comunidades/data_fuerza").createOrReplaceTempView("fuerza")

In [9]:
spark.sql('select * from seleccion').show()
spark.sql('select * from fuerza').show()

+--------------------+------------------+--------------+-----------+--------------------+------------+--------+--------------+---------+-----------------+
|            cont_rut|             score|total_pago_f29|   IVA_neto|     unidad_regional|n_documentos|lifetime|alerta_inicial|comunidad|      modularidad|
+--------------------+------------------+--------------+-----------+--------------------+------------+--------+--------------+---------+-----------------+
|AaF9igyV0h/q8dpZo...| 0.020976488828855|          null|   32894369|    SANTIAGO ORIENTE|        null|    null|             0|    18856|0.789362584526045|
|jQZMX1akk5R+/q6gn...|0.0413690502565515|          null|     175630|                null|        null|    null|             0|    24949|0.789362584526045|
|NE3RTDcWkhBInCQkt...|0.0192375917968015|           0.0|   55890027|   SANTIAGO PONIENTE|        null|   842.0|             0|    18856|0.789362584526045|
|Jn46LZRg9mBct7sSJ...|0.0413690502565515|          null|      79832|  

+--------------------+--------------------+-------+
|              emisor|            receptor|     Fi|
+--------------------+--------------------+-------+
|r6jpwMWW1oy67xtVw...|hGjfqdoNb4D1ohPfZ...| 7.0E-4|
|r6kSN7rfHzjPajlGI...|PPjo6zS/Y/K56iYy+...|  0.009|
|r6lMr0Rj+GGrhf1z9...|/b5pEez65E4yQLWjo...|0.02025|
|r6oipZZZKhWwgQ83/...|BoRC8ax6ves0Rcpru...|0.00115|
|r6oipZZZKhWwgQ83/...|LzUPAP9VR7rz3TaSr...| 9.0E-4|
|r6pHnsp2jbmcvHq8/...|SCDkYx8G56uXFkUIK...| 0.0076|
|r6qW9qzJ/AAtpjDeN...|s5swK8E8K1iCWcRF7...| 0.9359|
|r71vxsrmmvwmeH+NO...|5XQ428wO0WzRta2Qj...| 9.0E-4|
|r72FBdRZOOokZd50m...|Pb+o6zrpG+PBuzJj9...| 1.5E-4|
|r735QCrETnL9yJyY2...|i71/E5q/sy5mRyby8...| 3.0E-4|
|r76nddTkysw3yu1ya...|2aZuDpDpUevC56ODx...| 0.0078|
|r76nddTkysw3yu1ya...|F3DmeA7JYFZju18Zi...| 0.0089|
|r76nddTkysw3yu1ya...|WGqK6fleeoTWZIe46...| 0.0019|
|r76nddTkysw3yu1ya...|Ysfj8d5wmP69Y7vDQ...| 9.0E-4|
|r76nddTkysw3yu1ya...|gnDS2RCpCx6xFNPgL...| 0.0063|
|r76nddTkysw3yu1ya...|kV0LzS5JajliJdUnh...|0.00155|
|r76nddTkysw

In [10]:
## Seleciconamos las relaciones que onvolucran a esos contribuyentes (arcos)

In [11]:
spark.sql('select emisor, receptor, Fi,cont_rut as cont_rut1 from fuerza left join seleccion on fuerza.emisor=seleccion.cont_rut').createOrReplaceTempView("fuerza")

In [12]:
spark.sql('select emisor, receptor, Fi,cont_rut1, cont_rut as cont_rut2 from fuerza left join seleccion on fuerza.receptor=seleccion.cont_rut').createOrReplaceTempView("fuerza")

In [13]:
spark.sql("select emisor, receptor, Fi from fuerza where (cont_rut1 is not null and cont_rut2 is not null)").createOrReplaceTempView("fuerza")

In [14]:
fuerza=spark.sql('select * from fuerza')
fuerza=fuerza.toPandas()
#guardamos los valores de contaminacion con las columnas adicionales
fuerza.to_csv('/home/cdsw/data/processed/muestra_visualizacion/fuerza_IVA_representante_FILTERED.csv', index=False)
